In [2]:
import sys; sys.path.insert(0,'../../..')
from corpus_graph import *
setup_log(to_screen=False)

In [3]:
CG = CorpusGraph('markmark,txtlab,chadwyck,gildedage')
# g = CG.graph(only_identical=True)

[MarkMark] Iterating texts:   0%|          | 0/355 [00:00<?, ?it/s]

[TxtLab] Iterating texts:   0%|          | 0/450 [00:00<?, ?it/s]

[Chadwyck] Iterating texts:   0%|          | 0/1337 [00:00<?, ?it/s]

[GildedAge] Iterating texts:   0%|          | 0/334 [00:00<?, ?it/s]

In [16]:
def nodes_of_type(self,typename='text', g=None, **kwargs):
    if not g: g=self.graph(**kwargs)
    for node in g.nodes():
        if g.nodes[node].get('node_type') == typename:
            yield node

def text_nodes(self,g=None,**kwargs):
    yield from nodes_of_type(self,g=g,typename='text')

def graph_identity(self,**kwargs):
    return self.graph(only_identical=True,**kwargs)

def graph_identity_matches(self,g=None,**kwargs):
    if not g: g=graph_identity(self,**kwargs)
    return g

def components_identity(self,g=None,**kwargs):
    if not g: g=graph_identity(self,**kwargs)
    yield from nx.connected_components(g)

def graph_identity_texts(self,g=None,**kwargs):
    g = graph_identity(self,g=g,**kwargs)
    gnew=nx.Graph()
    for connected_nodes in components_identity(self,g=g):
        connected_nodes_text = [node for node in connected_nodes if g.nodes[node].get('node_type')=='text']
        for node in connected_nodes_text:
            if not gnew.has_node(node): gnew.add_node(node,**g.nodes[node])
        edges=[(n1,n2) for n1 in connected_nodes_text for n2 in connected_nodes_text if n1<n2]
        rel='_is'
        for n1,n2 in edges: gnew.add_edge(n1,n2,rel=rel)
    
    return gnew

def iter_identity_texts(self,g=None,**kwargs):
    if not g: g = graph_identity_texts(self,g=g,**kwargs)
    
    nodes=set()
    for node in get_tqdm(list(g.nodes())):
        node_text = Text(node)
        nodes|={node_text}
        for neighb in g.neighbors(node):
            neighb_text = Text(neighb)
            nodes|={neighb_text}
            node_text._is|={neighb_text}
            neighb_text._is|={node_text}
    
    yield from nodes


In [28]:
t=Text('_markmark/Wells_H._G..The_War_of_the_Worlds')
t

[TextMarkMark](_markmark/Wells_H._G..The_War_of_the_Worlds)

In [30]:
t.wikidata.meta

{'_addr': '_wikidata/Q213019',
 '_addr_source': '_markmark/Wells_H._G..The_War_of_the_Worlds',
 'author': 'H.G. Wells',
 'corpus': 'MarkMark',
 'dob': 1866,
 'dod': 1946,
 'gender': 'M',
 'name_first': 'Herbert',
 'name_last': 'Wells',
 'nation': 'English',
 'num_words': 67024,
 'ocr_accuracy': 0,
 'title': 'The War of the Worlds',
 'year': 1898,
 '_addr_wikidata': '_wikidata/Q213019',
 'from_text_func': 1,
 'name_middle': 'George',
 'id': 'Q213019',
 'award_received': ['NPR_Top_100_Science_Fiction_and_Fantasy_Books',
  "Le_Monde's_100_Books_of_the_Century"],
 'biblioteca_nacional_de_espana_id': 'XX2230813',
 'commons_category': 'The War of the Worlds (1898)',
 'copyright_status': ['public_domain', 'public_domain'],
 'country_of_origin': 'United Kingdom',
 'derivative_work': ['The_War_of_the_Worlds',
  'War_of_the_Worlds',
  'War_of_the_Worlds',
  'The_War_of_the_Worlds',
  'War_of_the_Worlds_2:_The_Next_Wave',
  "H._G._Wells'_War_of_the_Worlds",
  'The_War_of_the_Worlds',
  'War_of_th

In [20]:
nodes = list(iter_identity_texts(CG))
nodes

[CorpusGraph] Iterating texts:   0%|          | 0/2476 [00:00<?, ?it/s]

  0%|          | 0/2765 [00:00<?, ?it/s]

[[TextChadwyck](_chadwyck/Early_American_Fiction/herberth.09),
 [TextChadwyck](_chadwyck/Eighteenth-Century_Fiction/fieldinh.01),
 [TextMarkMark](_markmark/Brown_Dan.The_Lost_Symbol),
 [TextGildedAge](_gildedage/1876.Stowe.Am.M.Betty_S_Bright_Idea_Deacon_Pitkin_S_Farm_And_The_First_Christmas),
 [TextGildedAge](_gildedage/1905.Chesnutt.Am.M.The_Colonel_S_Dream),
 [TextChadwyck](_chadwyck/Early_American_Fiction/myerspha.05),
 [TextChadwyck](_chadwyck/Eighteenth-Century_Fiction/fieldins.01),
 [TextMarkMark](_markmark/Golding_William.Lord_of_the_Flies),
 [TextGildedAge](_gildedage/1900.Wister.Am.M.The_Jimmyjohn_Boss_And_Other_Stories),
 [TextChadwyck](_chadwyck/Early_American_Fiction/trowbrid.07),
 [TextChadwyck](_chadwyck/Early_American_Fiction/ingraham.60),
 [TextWikidata](_wikidata/Q3878987),
 [TextTxtLab](_txtlab/DE_1914_KafkaFranz_Amerika_Novel),
 [TextChadwyck](_chadwyck/Early_American_Fiction/browncha.16),
 [TextTxtLab](_txtlab/EN_1884_LyallEdna_WeTwo_Novel),
 [TextChadwyck](_chadwy

In [26]:
goethe_sorrows = Text('DE_1774_Goethe,Johann_DieLeidendesjungenWerthers1_Novel', 'txtlab')
goethe_sorrows.meta

{'_addr': '_txtlab/DE_1774_GoetheJohann_DieLeidendesjungenWerthers1_Novel',
 'id': 'DE_1774_Goethe,Johann_DieLeidendesjungenWerthers1_Novel'}

In [18]:
g = graph_identity_texts(CG)
list(g.neighbors('_wikidata/Q213019'))

[CorpusGraph] Iterating texts:   0%|          | 0/2476 [00:00<?, ?it/s]

['_markmark/Wells_H._G..The_War_of_the_Worlds']

In [7]:
Text('_wikidata/Q213019').meta

{'_addr': '_wikidata/Q213019',
 'author': 'H.G. Wells',
 'award_received': ['NPR_Top_100_Science_Fiction_and_Fantasy_Books',
  "Le_Monde's_100_Books_of_the_Century"],
 'biblioteca_nacional_de_espana_id': 'XX2230813',
 'commons_category': 'The War of the Worlds (1898)',
 'copyright_status': ['public_domain', 'public_domain'],
 'country_of_origin': 'United Kingdom',
 'derivative_work': ['The_War_of_the_Worlds',
  'War_of_the_Worlds',
  'War_of_the_Worlds',
  'The_War_of_the_Worlds',
  'War_of_the_Worlds_2:_The_Next_Wave',
  "H._G._Wells'_War_of_the_Worlds",
  'The_War_of_the_Worlds',
  'War_of_the_Worlds'],
 'encyclopaedia_britannica_online_id': 'topic/The-War-of-the-Worlds-novel-by-Wells',
 'fantlab_work_id': '9457',
 'first_line': 'No one would have believed in the last years of the nineteenth century that this world was being watched keenly and closely by intelligences greater than man’s and yet as mortal as his own; that as men busied themselves about their various concerns they were

In [12]:
# for node in iter_identity_texts(CG):
#     if len(node._is):
#         print(node)
#         for n in node._is:
#             print(f' ...is also: {n}')
#         print()


In [28]:
t=Text('_markmark/Faulkner_William.The_Sound_and_the_Fury')
t.metadata(force=True)

{'_addr': '_markmark/Faulkner_William.The_Sound_and_the_Fury',
 'author': 'Faulkner, William',
 'corpus': 'MarkMark',
 'dob': 1897,
 'dod': 1962,
 'gender': 'M',
 'name_first': 'William',
 'name_last': 'Faulkner',
 'nation': 'American',
 'num_words': 112789,
 'ocr_accuracy': 0,
 'title': 'The Sound and the Fury',
 'year': 1929,
 '_addr_wikidata': '_wikidata/Q135515',
 'from_text_func': 1,
 'name_middle': 'Cuthbert',
 'id': 'Faulkner_William.The_Sound_and_the_Fury',
 '_id_wikidata': 'Q135515'}

In [25]:
t

[TextWikidata](_wikidata/Q135515)

In [7]:
g.add_edge(1,2,x=10,y=20)

0

In [10]:
g.get_edge_data(1,2)

{0: {'x': 10, 'y': 20}}